# Add Configuration attribute to Teleview

This notebook Helps you add a configuration attribute to the Teleview Project.

So far it works only for adding boolean,string, int, double config parameters


## TODO

Feature | Comments | Status
:-----  | :---------- | :-----
jpeg_Qualtiy: value | |Done
WhiteBalance: value | |Done
Brightness: value   | |Done
/sendVideo ?        | |
/sendAudio ?        | Integrate I2S Microphone Request Audio  |
CVMotionDetect      |  motion detect by Image not PIR  ON/OFF  | (Almost)
faceDetect          |  ON/OFF and face recognize then add to Photo Caption. https://github.com/Xinyuan-LilyGo/esp32-face-recognition|
send-email          | email,server,token..etc | DONE
motionDetect Active times| define times where motion detection is enabled. |
send photo at:      | sunrise, noon, afternoon, late afternoon, sunset. with offset time option | 
GETGEOlOCATION      | send location telegram message to the bot | 
NTP syncing         |  Requires NTP syncing. uses time zone ...etc.  | DONE
MQTT                | integrate MQTT: on all events ,send an MQTT message |
Product WebSite     | build a website to host the User's Guide, Registration & ecommerce |
JavaScript Validation| in the options page, have the elements more interactive and dimm inputs that are possible. example: send email checkbox is only enabled, when all the other email settings entered |
M5 Camera-TimerX/F Battery & RTC features | Integrate M5 Camera-Timer X&F Battery and RTC API |
use "src/U8x8lib.h" for LCD |  use a differnt library for the LCD display: #include "src/U8x8lib.h" https://robotzero.one/ttgo-security-camera-pir/ |


better Config saving|https://hieromon.github.io/AutoConnect/achandling.html#saving-autoconnectelements-with-json|
Java Script validation of Input | https://hieromon.github.io/AutoConnect/achandling.html#using-javascript |  
  

## Setup Variables

In [10]:
persist_h=r'..\persist.h'
telegramUtils_h=r'..\telegram_utils.h'
webPages_h=r'..\webPages.h'

# strings will be added after this attribute
# anchor_attribute="vFlip"

boolean_anchor_attribute="// [:ADD BOOLEAN HERE:]"
int_anchor_attribute="// [:ADD INT HERE:]"
string_anchor_attribute="// [:ADD STRING HERE:]"

In [41]:
import re 

def returnMatch(theHash, new_attribute_type, line):
    line=line.strip()
    match = line.startswith(F"{boolean_anchor_attribute}{theHash}")
    if new_attribute_type=="boolean" :
        match = line.startswith(F"{boolean_anchor_attribute}{theHash}")
    elif new_attribute_type=="String" :
        match = line.startswith(F"{string_anchor_attribute}{theHash}")
    elif new_attribute_type=="int" :
        match =line.startswith(F"{int_anchor_attribute}{theHash}")
    elif new_attribute_type=="double" :
        match = line.startswith(F"{int_anchor_attribute}{theHash}")
    if (match):
        print (F"theHash:{theHash}, new_attribute_type:{new_attribute_type}, match:{match}, line:{line}")
    
    return(match)

## Changes in persist.h

In [18]:


def changePersist(new_attribute_name, new_attribute_type, new_attribute_desc):
    with open(persist_h,"r") as persistH:
        lines=[]
        for line in persistH:            
            ############################
            
            theHash="#1" # struct config_item
            match1 = returnMatch(theHash,new_attribute_type,line)
                
            theHash="#2" # configItems
            match2 = returnMatch(theHash,new_attribute_type,line)
            
            #theHash="#3" # remove/delete
            #match3 = returnMatch(theHash,new_attribute_type,line)

            theHash="#4" # loadConfiguration
            match4 = returnMatch(theHash,new_attribute_type,line)

            theHash="#5" # saveConfiguration
            match5 = returnMatch(theHash,new_attribute_type,line)
            
            theHash="#6" # printConfiguration
            match6 = returnMatch(theHash,new_attribute_type,line)
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            if match1:
                new_line=F"  {new_attribute_type} {new_attribute_name};"
            if match2:
                if new_attribute_type=="boolean" :
                    new_line=F"  .{new_attribute_name} = true,"
                elif new_attribute_type=="String" :
                    new_line=F"  .{new_attribute_name} = \"\","
                elif new_attribute_type in ("int","double") :
                    new_line=F"  .{new_attribute_name} = 0,"

            #if match3:
            #    new_line=F"    prefs.remove(\"{new_attribute_name}\");"

            if match4:
                if new_attribute_type=="boolean" :
                    new_line=F"    ci.{new_attribute_name} = prefs.getBool(\"{new_attribute_name}\",configItems.{new_attribute_name});"
                elif new_attribute_type=="String" :
                    new_line=F"    ci.{new_attribute_name} = prefs.getString(\"{new_attribute_name}\",configItems.{new_attribute_name});"
                elif new_attribute_type=="int" :
                    new_line=F"    ci.{new_attribute_name} = prefs.getInt(\"{new_attribute_name}\",configItems.{new_attribute_name});"
                elif new_attribute_type=="double" :
                    new_line=F"    ci.{new_attribute_name} = prefs.getDouble(\"{new_attribute_name}\",configItems.{new_attribute_name});"

            if match5:
                if new_attribute_type=="boolean" :
                    new_line=F'      {{ prefs.putBool("{new_attribute_name}",ci->{new_attribute_name}); }}'
                elif new_attribute_type=="String" :
                    new_line =F'      {{ prefs.putString("{new_attribute_name}",ci->{new_attribute_name}); }}'
                elif new_attribute_type=="int" :
                    new_line =F'      {{ prefs.putInt("{new_attribute_name}",ci->{new_attribute_name}); }}'
                elif new_attribute_type=="double" :
                    new_line =F'      {{ prefs.putDouble("{new_attribute_name}",ci->{new_attribute_name}); }}'

            if match6:
                if new_attribute_type=="boolean" :
                    new_line =F'  result += prefix+"{new_attribute_name}   "+sep+"";'
                    new_line2=F'  result += (ci->{new_attribute_name} ? String("true") : String("false"))  + suffix;'
                elif new_attribute_type in ("String","int","double") :
                    new_line =F'  result += prefix+"{new_attribute_name}   "+sep+"";'
                    new_line2=F'  result += String(ci->{new_attribute_name})  + suffix;'
                    
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            ############################
        new_persist_h=open(F"{persist_h}_new","w")
        for line in lines:
            #print (line)
            new_persist_h.write(line)
        new_persist_h.close()

    if os.path.exists(F"{persist_h}_orig"):
        os.unlink(F"{persist_h}_orig")
    os.rename(persist_h,F"{persist_h}_orig")
    os.rename(F"{persist_h}_new",persist_h)

## Changes in telegram_utils.h

In [19]:
import re 
import os

def changeTelegram_utils(new_attribute_name, new_attribute_type, new_attribute_desc):
    with open(telegramUtils_h,"r") as telegramUtilsH:
        lines=[]
        for line in telegramUtilsH:
            #print (line)
            ############################
            
            theHash="#1"
            match1 = returnMatch(theHash,new_attribute_type,line)

            theHash="#2"
            match2 = returnMatch(theHash,new_attribute_type,line)

            theHash="#3"
            match3 = returnMatch(theHash,new_attribute_type,line)
            
            theHash="#4"
            match4 = returnMatch(theHash,new_attribute_type,line)
            
            heHash="#5"
            match5 = returnMatch(theHash,new_attribute_type,line)
            
            heHash="#6"
            match6 = returnMatch(theHash,new_attribute_type,line)
            
            heHash="#7"
            match7 = returnMatch(theHash,new_attribute_type,line)
            
            heHash="#8"
            match8 = returnMatch(theHash,new_attribute_type,line)
            
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            new_line3=""
            if match1:
                if new_attribute_type=="boolean" :
                    new_line =F'\n  keyboardJson += R"([{{ "text" : "{new_attribute_desc} )";'
                    new_line2=F'  keyboardJson += (configItems.{new_attribute_name}?"ON\\u2705":"OFF\\u274C");'
                    new_line3=F'  keyboardJson += R"(", "callback_data" : "/{new_attribute_name}" }}],)";'
            
            if match2:
                if new_attribute_type=="boolean" :
                    new_line =F'          }}else if (text == "/{new_attribute_name}") {{'
                    new_line2=F'            configItems.{new_attribute_name} = !configItems.{new_attribute_name};'
                    
            if match3:
                new_line =F'          welcome += "\\t {new_attribute_name} | {new_attribute_desc}\\n";'
                
            if match4:
                new_line =F'          welcome += "\\t  ,MODE_SET_{new_attribute_name}=XX \\n";'
            
            if match5:
                new_line  =F'          if (bTeleAnsMode==MODE_SET_{new_attribute_name}) { \\n";'
                new_line2 =F'          bTeleAnsMode=MODE_SET_NONE; \\n";'
                new_line3 =F'          configItems.{new_attribute_name}=({new_attribute_type}) text.toInt(); \} \\n";'
                
            if match6:
                new_line  =F'          else if(text == "/set{new_attribute_name}") \{ \\n";'
                new_line2 =F'          configItems.{new_attribute_name}=0; \\n";'
                new_line3 =F'          bot.sendMessage(chat_id, "Please insert {new_attribute_desc}", ""); \\n";'
                new_line4 =F'          bTeleAnsMode=MODE_SET_{new_attribute_name}; \\n";'
                new_line5 =F'          bPrintOptions=false; \} \\n";'
            
            if match7:
                new_line  =F'          lkeyboardJson += ",[\"/set{new_attribute_name}\""; \\n";'

            if match8:
                new_line  =F'          welcome += "/set{new_attribute_name}    | Sets the {new_attribute_description}\n"; \\n";'
                
                        
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            if new_line3!="":
                print (new_line3)
                lines.append(new_line3+"\n")
            ############################
        new_telegramUtils_h=open(F"{telegramUtils_h}_new","w")
        for line in lines:
            #print (line)
            new_telegramUtils_h.write(line)
        new_telegramUtils_h.close()
        
    if os.path.exists(F"{telegramUtils_h}_orig"):
        os.unlink(F"{telegramUtils_h}_orig")
    os.rename(telegramUtils_h,F"{telegramUtils_h}_orig")
    os.rename(F"{telegramUtils_h}_new",telegramUtils_h)

## Changes in webPages.h

In [21]:
import re 

def changeWebPages(new_attribute_name, new_attribute_type, new_attribute_desc):

    with open(webPages_h,"r") as webPagesH:
        lines=[]
        for line in webPagesH:
            #print (line)
            ############################
            theHash="#1"
            match1 = returnMatch(theHash,new_attribute_type,line)

            theHash="#2"
            match2 = returnMatch(theHash,new_attribute_type,line)


            theHash="#3"
            match3 = returnMatch(theHash,new_attribute_type,line)
            #lookfor=r'{ "name": "XvFlip","type": "ACCheckbox","value": "","labelPosition": "AC_Infront" ,"label": "Vertical Flip","checked": false,"global": true},'
            #lookfor=lookfor.replace("vFlip",anchor_attribute)
            #match3 = re.search(re.escape(lookfor), line)
                
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            new_line3=""
            if match1:
                if new_attribute_type=="boolean" :
                    new_line =F'    configItems.{new_attribute_name}=(args.hasArg("X{new_attribute_name}")?true:false);'
                elif new_attribute_type in ("String") :
                    new_line =F'    configItems.{new_attribute_name}=(args.hasArg("X{new_attribute_name}")?args.arg("X{new_attribute_name}"):"");'
                elif new_attribute_type in ("int") :
                    new_line =F'    configItems.{new_attribute_name}=(args.hasArg("X{new_attribute_name}")?args.arg("X{new_attribute_name}").toInt():0);'    
                elif new_attribute_type in ("double") :
                    new_line =F'    configItems.{new_attribute_name}=(args.hasArg("X{new_attribute_name}")?args.arg("X{new_attribute_name}").toDouble():0);'

            if match2:
                if new_attribute_type=="boolean" :
                    new_line =F'    aux["X{new_attribute_name}"].as<AutoConnectCheckbox>().checked=configItems.{new_attribute_name};'
                elif new_attribute_type in ("String","int","double") :
                    new_line =F'    aux["X{new_attribute_name}"].as<AutoConnectInput>().value=configItems.{new_attribute_name};'

            if match3:
                if new_attribute_type=="boolean" :
                    new_line =F'    {{ "name": "X{new_attribute_name}", "type": "ACCheckbox", "value": "", "labelPosition": "AC_Infront" , "label": "{new_attribute_desc}", "checked": false, "global": true }},'
                elif new_attribute_type=="String" :
                    new_line =F'    {{ "name": "X{new_attribute_name}", "type": "ACInput", "value": "", "labelPosition": "AC_Infront" , "label": "{new_attribute_desc}","global": true , "placeholder": "{new_attribute_name}",  "pattern": "^(.+)$"}},'
                elif new_attribute_type=="int" :
                    new_line =F'    {{ "name": "X{new_attribute_name}", "type": "ACInput", "value": "", "labelPosition": "AC_Infront" , "label": "{new_attribute_desc}","global": true , "placeholder": "{new_attribute_name}",  "pattern": "^([+-]?([0-9]*))$"}},'
                elif new_attribute_type=="double" :
                    new_line =F'    {{ "name": "X{new_attribute_name}", "type": "ACInput", "value": "", "labelPosition": "AC_Infront" , "label": "{new_attribute_desc}","global": true , "placeholder": "{new_attribute_name}",  "pattern": "^([+-]?([0-9]*[.])?[0-9]+)$"}},'
            
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            if new_line3!="":
                print (new_line3)
                lines.append(new_line3+"\n")
            ############################
        new_webPages_h=open(F"{webPages_h}_new","w")
        for line in lines:
            #print (line)
            new_webPages_h.write(line)
        new_webPages_h.close()

    if os.path.exists(F"{webPages_h}_orig"):
        os.unlink(F"{webPages_h}_orig")
    os.rename(webPages_h,F"{webPages_h}_orig",)
    os.rename(F"{webPages_h}_new",webPages_h)

{
  "name": "date",
  "type": "ACElement",
  "value": "<label for='picker'>Date:</label><input type='date' id='picker' name='date'>"
}

https://hieromon.github.io/AutoConnect/achandling.html#place-the-input-elements-within-a-form


<input type='date' id='picker' name='FromDate'>

In [ ]:
"""
    ,{
        new_attribute_name: "ftpServer",
        new_attribute_type: "boolean",
        new_attribute_desc: "Enable FTP server",
    }
    ,{
        new_attribute_name: "faceDetect",
        new_attribute_type: "boolean",
        new_attribute_desc: "Alert on detecting a face",
    }
    ,{
        new_attribute_name: "takeVideo",
        new_attribute_type: "boolean",
        new_attribute_desc: "Take a Video",
    }
    ,{
        new_attribute_name: "jpegQulaity",
        new_attribute_type: "int",
        new_attribute_desc: "Jpeg Quality 1-62[10-12]",
    }
    ,{
        new_attribute_name: "brightness",
        new_attribute_type: "int",
        new_attribute_desc: "PhotoBrightness 1-62[10-12]",
    }
    ,{
        new_attribute_name: "whiteBalance",
        new_attribute_type: "int",
        new_attribute_desc: "PhotoBrightness 1-62[10-12]",
    }
    ,{
        new_attribute_name: "sendEmail",
        new_attribute_type: "boolean",
        new_attribute_desc: "Alert by Email.",
    }
    ,{
        new_attribute_name: "adminEmail",
        new_attribute_type: "String",
        new_attribute_desc: "Email 1-dont use your real email, create a new one",
    }
    ,{
        new_attribute_name: "userEmail",
        new_attribute_type: "String",
        new_attribute_desc: "Email 2.",
    }
    ,{
        new_attribute_name: "sMTPServer",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email Server",
    }
    ,{
        new_attribute_name: "sMTPUsername",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email username",
    }
    ,{
        new_attribute_name: "sMTPPassword",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email Password",
    }
    ,{
        new_attribute_name: "sMTPPort",
        new_attribute_type: "int",
        new_attribute_desc: "SMTP Email Port",
    }
    ,{
        new_attribute_name: "sMTPTLS",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP TLS/SSL Required",
    }
    ,{
        new_attribute_name: "motionDetectFrom",
        new_attribute_type: "int",
        new_attribute_desc: "MotionDetect active from",
    }
    ,{
        new_attribute_name: "motionDetectTo",
        new_attribute_type: "int",
        new_attribute_desc: "MotionDetect active from",
    }
    
    ,{
        new_attribute_name: "longitude",
        new_attribute_type: "double",
        new_attribute_desc: "Location Longitude",
    }
    ,{
        new_attribute_name: "latitude",
        new_attribute_type: "double",
        new_attribute_desc: "Location Latitude",
    }
    ,{
        new_attribute_name: "altitude",
        new_attribute_type: "double",
        new_attribute_desc: "Location Altitude",
    }
    ,{
        new_attribute_name: "snapSunrise",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Sunrise",
    }
    ,{
        new_attribute_name: "snapNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Noon",
    }
    ,{
        new_attribute_name: "snapAfterNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on After Noon",
    }
    ,{
        new_attribute_name: "snapLateAfterNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Late After Noon",
    }
    ,{
        new_attribute_name: "snapSunset",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Sunset",
    }
    ,{
        new_attribute_name: "snapEvening",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Evening",
    }
    
    {
        'new_attribute_name': "cvChangePercent",
        'new_attribute_type': "int",
        'new_attribute_desc': "CV Motion Detect Percent change in number of DarkPixels:"
    },
    {
        'new_attribute_name': "cvIntervalSec",
        'new_attribute_type': "int",
        'new_attribute_desc': "CV Motion Detect interval between each 2 consecutive photo capture:"
    },
"""

## Assembly : add multiple attributes

In [42]:

attsToAdd = [ 
    {
        'new_attribute_name': "cvChangePercent",
        'new_attribute_type': "int",
        'new_attribute_desc': "CV Motion Detect Percent change in number of DarkPixels:"
    },
    {
        'new_attribute_name': "cvIntervalSec",
        'new_attribute_type': "int",
        'new_attribute_desc': "CV Motion Detect interval between each 2 consecutive photo capture:"
    },
]

for att in attsToAdd:
    print(att)
    changePersist         (att['new_attribute_name'],att['new_attribute_type'],att['new_attribute_desc'])
    changeTelegram_utils  (att['new_attribute_name'],att['new_attribute_type'],att['new_attribute_desc'])
    changeWebPages        (att['new_attribute_name'],att['new_attribute_type'],att['new_attribute_desc'])

{'new_attribute_name': 'cvChangePercent', 'new_attribute_type': 'int', 'new_attribute_desc': 'CV Motion Detect Percent change in number of DarkPixels:'}
theHash:#1, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#1
  int cvChangePercent;
theHash:#2, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#2
  .cvChangePercent = 0,
theHash:#4, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#4
    ci.cvChangePercent = prefs.getInt("cvChangePercent",configItems.cvChangePercent);
theHash:#4, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#4
    ci.cvChangePercent = prefs.getInt("cvChangePercent",configItems.cvChangePercent);
theHash:#6, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#6
  result += prefix+"cvChangePercent   "+sep+"";
  result += String(ci->cvChangePercent)  + suffix;
theHash:#1, new_attribute_type:int, match:True, line:// [:ADD INT HERE:]#1
    configItems.cvChangePercent=(args.hasArg("XcvChangePercent")?args.arg("cvCh